In [12]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import requests

In [18]:
page = 'https://www.wunderground.com/history/daily/LEMD/date/'
request = requests.get(page)
soup = BS(request.content, 'html.parser')

In [19]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/Organization" lang="en" prefix="og: http://ogp.me/ns#"><head itemscope="" itemtype="http://schema.org/WebSite">
<meta charset="utf-8"/>
<title>Madrid, Spain Weather History | Weather Underground</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="//widgets.outbrain.com" rel="dns-prefetch"/>
<link href="//odb.outbrain.com" rel="dns-prefetch"/>
<link href="//c.amazon-adsystem.com" rel="dns-prefetch"/>
<link href="//s.amazon-adsystem.com" rel="dns-prefetch"/>
<link href="//aax.amazon-adsystem.com" rel="dns-prefetch"/>
<link href="//partner.googleadservices.com" rel="dns-prefetch"/>
<link href="//tpc.googlesyndication.com" rel="dns-prefetch"/>
<link href="//pagead2.googlesyndication.com" rel="dns-prefetch"/>
<link href="//h.nexac.com" rel="dns-prefetch"/>
<link href="//js-sec.indexww.com" rel="dns-

In [35]:
table = soup.find('dev', class_="observation-table ng-star-inserted")
#container = table.nextSibling
table


In [28]:
table = soup.find('div', class_="small-12 columns has-sidebar")
container = table.nextSibling.nextSibling
container

<div _ngcontent-sc317="" class="small-12 columns has-sidebar"><div _ngcontent-sc317="" class="row"><div _ngcontent-sc317="" class="columns small-12"><lib-city-history-summary _ngcontent-sc317="" _nghost-sc259=""><div _ngcontent-sc259=""><div _ngcontent-sc259="" class="summary-title">Summary</div><div _ngcontent-sc259="" class="summary-table"> No data recorded <!-- --><!-- --></div></div></lib-city-history-summary></div></div><div _ngcontent-sc317="" class="row is-sidebar ng-star-inserted"><div _ngcontent-sc317="" class="region-sidebar"><ad-wx-mid-300-var _ngcontent-sc317="" libsticky=""><div class="ad-wrap ad-desktop" id="ad-wx-mid-300-var-browser"><div id="WX_Mid300Variable"></div></div></ad-wx-mid-300-var><ad-mw-position-4 _ngcontent-sc317=""><div class="ad-wrap ad-mobile" id="position-4-banner-ad-browser"><div id="MW_Position4"></div></div></ad-mw-position-4></div></div><!-- --></div>

In [29]:
table = soup.find('div', class_="small-12 columns has-sidebar")
container = table.nextSibling.nextSibling.nextSibling
container

<div _ngcontent-sc317="" class="columns small-12"><ad-wx-hsd-leader _ngcontent-sc317=""><div class="leaderboard-banner ad-wrap ad-desktop" id="leaderboard-ad-2-browser"><div id="WX_HsdLeader"></div></div></ad-wx-hsd-leader></div>

In [30]:
table = soup.find('div', class_="small-12 columns has-sidebar")
container = table.nextSibling.nextSibling.nextSibling.nextSibling
container

<div _ngcontent-sc317="" class="small-12 columns has-sidebar"><div _ngcontent-sc317="" class="row"><div _ngcontent-sc317="" class="columns small-12"><lib-city-history-observation _ngcontent-sc317="" _nghost-sc258=""><div _ngcontent-sc258=""><div _ngcontent-sc258="" class="observation-title">Daily Observations</div> No Data Recorded <!-- --><!-- --></div></lib-city-history-observation></div></div><div _ngcontent-sc317="" class="row is-sidebar ng-star-inserted"><div _ngcontent-sc317="" class="region-sidebar"><ad-wx-bot-300 _ngcontent-sc317="" libsticky=""><div class="ad-wrap ad-desktop" id="wx-bot-300-browser"><div id="WX_Bot300"></div></div></ad-wx-bot-300></div></div><!-- --></div>

In [31]:
table = soup.find('div', class_="small-12 columns has-sidebar")
container = table.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling
container

In [1]:
import requests
import datetime as dt
import pandas as pd
import datetime as dt

In [2]:
import meteostat
from meteostat import Hourly
from meteostat import Point

In [1]:
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from functools import reduce
import pandas as pd
import time

In [36]:
def render_page(url):
    driver = webdriver.Chrome('chromedriver')
    driver.get(url)
    time.sleep(3)
    r = driver.page_source
    driver.quit()
    return r

In [3]:
def scraper(page, dates):
    output = pd.DataFrame()

    for d in dates:

        url = str(str(page) + str(d))

        r = render_page(url)

        soup = BS(r, "html.parser")
        container = soup.find('lib-city-history-observation')
        check = container.find('tbody')

        data = []

        for c in check.find_all('tr', class_='ng-star-inserted'):
            for i in c.find_all('td', class_='ng-star-inserted'):
                trial = i.text
                trial = trial.strip('  ')
                data.append(trial)

        if round(len(data) / 17 - 1) == 31:
            Temperature = pd.DataFrame([data[32:128][x:x + 3] for x in range(0, len(data[32:128]), 3)][1:],
                                       columns=['Temp_max', 'Temp_avg', 'Temp_min'])
            Dew_Point = pd.DataFrame([data[128:224][x:x + 3] for x in range(0, len(data[128:224]), 3)][1:],
                                     columns=['Dew_max', 'Dew_avg', 'Dew_min'])
            Humidity = pd.DataFrame([data[224:320][x:x + 3] for x in range(0, len(data[224:320]), 3)][1:],
                                    columns=['Hum_max', 'Hum_avg', 'Hum_min'])
            Wind = pd.DataFrame([data[320:416][x:x + 3] for x in range(0, len(data[320:416]), 3)][1:],
                                columns=['Wind_max', 'Wind_avg', 'Wind_min'])
            Pressure = pd.DataFrame([data[416:512][x:x + 3] for x in range(0, len(data[416:512]), 3)][1:],
                                    columns=['Pres_max', 'Pres_avg', 'Pres_min'])
            Date = pd.DataFrame(data[:32][1:], columns=data[:1])
            Precipitation = pd.DataFrame(data[512:][1:], columns=['Precipitation'])
            print(str(str(d) + ' finished!'))
        elif round(len(data) / 17 - 1) == 28:
            Temperature = pd.DataFrame([data[29:116][x:x + 3] for x in range(0, len(data[29:116]), 3)][1:],
                                       columns=['Temp_max', 'Temp_avg', 'Temp_min'])
            Dew_Point = pd.DataFrame([data[116:203][x:x + 3] for x in range(0, len(data[116:203]), 3)][1:],
                                     columns=['Dew_max', 'Dew_avg', 'Dew_min'])
            Humidity = pd.DataFrame([data[203:290][x:x + 3] for x in range(0, len(data[203:290]), 3)][1:],
                                    columns=['Hum_max', 'Hum_avg', 'Hum_min'])
            Wind = pd.DataFrame([data[290:377][x:x + 3] for x in range(0, len(data[290:377]), 3)][1:],
                                columns=['Wind_max', 'Wind_avg', 'Wind_min'])
            Pressure = pd.DataFrame([data[377:464][x:x + 3] for x in range(0, len(data[377:463]), 3)][1:],
                                    columns=['Pres_max', 'Pres_avg', 'Pres_min'])
            Date = pd.DataFrame(data[:29][1:], columns=data[:1])
            Precipitation = pd.DataFrame(data[464:][1:], columns=['Precipitation'])
            print(str(str(d) + ' finished!'))
        elif round(len(data) / 17 - 1) == 29:
            Temperature = pd.DataFrame([data[30:120][x:x + 3] for x in range(0, len(data[30:120]), 3)][1:],
                                       columns=['Temp_max', 'Temp_avg', 'Temp_min'])
            Dew_Point = pd.DataFrame([data[120:210][x:x + 3] for x in range(0, len(data[120:210]), 3)][1:],
                                     columns=['Dew_max', 'Dew_avg', 'Dew_min'])
            Humidity = pd.DataFrame([data[210:300][x:x + 3] for x in range(0, len(data[210:300]), 3)][1:],
                                    columns=['Hum_max', 'Hum_avg', 'Hum_min'])
            Wind = pd.DataFrame([data[300:390][x:x + 3] for x in range(0, len(data[300:390]), 3)][1:],
                                columns=['Wind_max', 'Wind_avg', 'Wind_min'])
            Pressure = pd.DataFrame([data[390:480][x:x + 3] for x in range(0, len(data[390:480]), 3)][1:],
                                    columns=['Pres_max', 'Pres_avg', 'Pres_min'])
            Date = pd.DataFrame(data[:30][1:], columns=data[:1])
            Precipitation = pd.DataFrame(data[480:][1:], columns=['Precipitation'])
            print(str(str(d) + ' finished!'))
        elif round(len(data) / 17 - 1) == 30:
            Temperature = pd.DataFrame([data[31:124][x:x + 3] for x in range(0, len(data[31:124]), 3)][1:],
                                       columns=['Temp_max', 'Temp_avg', 'Temp_min'])
            Dew_Point = pd.DataFrame([data[124:217][x:x + 3] for x in range(0, len(data[124:217]), 3)][1:],
                                     columns=['Dew_max', 'Dew_avg', 'Dew_min'])
            Humidity = pd.DataFrame([data[217:310][x:x + 3] for x in range(0, len(data[217:310]), 3)][1:],
                                    columns=['Hum_max', 'Hum_avg', 'Hum_min'])
            Wind = pd.DataFrame([data[310:403][x:x + 3] for x in range(0, len(data[310:403]), 3)][1:],
                                columns=['Wind_max', 'Wind_avg', 'Wind_min'])
            Pressure = pd.DataFrame([data[403:496][x:x + 3] for x in range(0, len(data[403:496]), 3)][1:],
                                    columns=['Pres_max', 'Pres_avg', 'Pres_min'])
            Date = pd.DataFrame(data[:31][1:], columns=data[:1])
            Precipitation = pd.DataFrame(data[496:][1:], columns=['Precipitation'])
            print(str(str(d) + ' finished!'))
        else:
            print('Data not in normal length')

        dfs = [Date, Temperature, Dew_Point, Humidity, Wind, Pressure, Precipitation]

        df_final = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)

        df_final['Date'] = str(d) + "-" + df_final.iloc[:, :1].astype(str)

        output = output.append(df_final)

    print('Scraper done!')

    output = output[['Temp_avg', 'Temp_min', 'Dew_max', 'Dew_avg', 'Dew_min', 'Hum_max',
                     'Hum_avg', 'Hum_min', 'Wind_max', 'Wind_avg', 'Wind_min', 'Pres_max',
                     'Pres_avg', 'Pres_min', 'Precipitation', 'Date']]

    return output

In [6]:
dates = ['2015-1-1','2015-1-31']
page = 'https://www.wunderground.com/history/daily/LEMD/date/'
df_output = scraper(page,dates)

r = render_page(url)

In [66]:
def extract_table(table):
    day = [[] for i in range(10)]
    for row in table.find_all('tr', class_='ng-star-inserted'):
        for i, col in enumerate(row.find_all('td', class_='ng-star-inserted')):
            day[i].append(col.text)
    return day

In [67]:
import datetime as dt

## Pull Data

In [ ]:
seville = https://www.wunderground.com/history/daily/LEZL/date/2022-1-25
madrid = https://www.wunderground.com/history/daily/LEMD/date/2022-1-25
barcelona = https://www.wunderground.com/history/daily/LEBL/date/2022-1-25
bilbao = https://www.wunderground.com/history/daily/LEBB/date/2022-1-25
valencia = https://www.wunderground.com/history/daily/LEVC/date/2022-1-25

In [311]:
city_codes = {'LEZL':'seville'}
dates_stamp = pd.date_range(start=dt.datetime(2019,9,10), end=dt.datetime(2021,12,31),freq='D')
dates = [str(x)[:10] for x in dates_stamp]
city_data = {}
for code in city_codes.keys():
    data_1 = {}
    for date in dates:
        # Request page
        url = f'https://www.wunderground.com/history/daily/{code}/date/{date}'
        r = render_page(url)
        soup = BS(r, "html.parser")
        container = soup.find('lib-city-history-observation')
        table = container.find('tbody')

        # Extract table from page and add it to data
        data_1[date] = extract_table(table)
    city_data[code] = data_1

dates_ =[]
temps = []
dew_points = []
humidities = []
winds = []
wind_speeds = []
wind_gusts = []
pressures = []
precips = []
conditions = []
for day_key in data_1.keys():
    for ind, t in enumerate(data_1[day_key][0]):
        dates_.append(day_key+' '+data_1[day_key][0][ind])
        temps.append(data_1[day_key][1][ind])
        dew_points.append(data_1[day_key][2][ind])
        humidities.append(data_1[day_key][3][ind])
        winds.append(data_1[day_key][4][ind])
        wind_speeds.append(data_1[day_key][5][ind])
        wind_gusts.append(data_1[day_key][6][ind])
        pressures.append(data_1[day_key][7][ind])
        precips.append(data_1[day_key][8][ind])
        conditions.append(data_1[day_key][9][ind])
seville_end = pd.DataFrame({'date':dates_,
              'temp':temps,
              'dew_point':dew_points,
              'humidities': humidities,
              'wind':winds,
              'wind_speeds':wind_speeds,
              'pressures':pressures,
              'precips':precips,
              'condition':conditions})
seville_end.to_csv('../data/seville_end')

In [319]:
city_codes = {'LEZL':'seville',
              'LEBL':'barcelona',
              'LEBB':'bilboa',
              'LEVC':'valencia'}
dates_stamp = pd.date_range(start=dt.datetime(2015,5,2), end=dt.datetime(2021,12,31),freq='D')
dates = [str(x)[:10] for x in dates_stamp]
city_data = {}

In [ ]:
city_codes = {'LEBB':'bilboa'}
dates_stamp = pd.date_range(start=dt.datetime(2018,5,18), end=dt.datetime(2021,12,31),freq='D')
dates = [str(x)[:10] for x in dates_stamp]
city_data = {}
for code in city_codes.keys():
    data_1 = {}
    for date in dates:
        # Request page
        url = f'https://www.wunderground.com/history/daily/{code}/date/{date}'
        r = render_page(url)
        soup = BS(r, "html.parser")
        container = soup.find('lib-city-history-observation')
        table = container.find('tbody')

        # Extract table from page and add it to data
        data_1[date] = extract_table(table)
    city_data[code] = data_1

In [340]:
data_1.keys()

dict_keys(['2015-05-02', '2015-05-03', '2015-05-04', '2015-05-05', '2015-05-06', '2015-05-07', '2015-05-08', '2015-05-09', '2015-05-10', '2015-05-11', '2015-05-12', '2015-05-13', '2015-05-14', '2015-05-15', '2015-05-16', '2015-05-17', '2015-05-18', '2015-05-19', '2015-05-20', '2015-05-21', '2015-05-22', '2015-05-23', '2015-05-24', '2015-05-25', '2015-05-26', '2015-05-27', '2015-05-28', '2015-05-29', '2015-05-30', '2015-05-31', '2015-06-01', '2015-06-02', '2015-06-03', '2015-06-04', '2015-06-05', '2015-06-06', '2015-06-07', '2015-06-08', '2015-06-09', '2015-06-10', '2015-06-11', '2015-06-12', '2015-06-13', '2015-06-14', '2015-06-15', '2015-06-16', '2015-06-17', '2015-06-18', '2015-06-19', '2015-06-20', '2015-06-21', '2015-06-22', '2015-06-23', '2015-06-24', '2015-06-25', '2015-06-26', '2015-06-27', '2015-06-28', '2015-06-29', '2015-06-30', '2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-

In [249]:
day_key = '2021-09-16'
city_key = list(city_data.keys())[0]
day_key = list(city_data[city_data_key].keys())[0]
city_data[city_key][day_key]

[['11:00 PM',
  '11:30 PM',
  '12:00 AM',
  '12:30 AM',
  '1:00 AM',
  '1:30 AM',
  '2:00 AM',
  '2:30 AM',
  '3:00 AM',
  '3:30 AM',
  '4:00 AM',
  '4:30 AM',
  '5:00 AM',
  '5:30 AM',
  '6:00 AM',
  '6:30 AM',
  '7:00 AM',
  '7:30 AM',
  '8:00 AM',
  '8:30 AM',
  '9:00 AM',
  '9:30 AM',
  '10:00 AM',
  '10:30 AM',
  '11:00 AM',
  '11:30 AM',
  '12:00 PM',
  '12:30 PM',
  '1:00 PM',
  '1:30 PM',
  '2:00 PM',
  '2:30 PM',
  '3:00 PM',
  '3:30 PM',
  '4:00 PM',
  '4:30 PM',
  '5:00 PM',
  '5:30 PM',
  '6:00 PM',
  '6:30 PM',
  '7:00 PM',
  '7:30 PM',
  '8:00 PM',
  '8:30 PM',
  '9:00 PM',
  '9:30 PM',
  '10:00 PM',
  '10:30 PM'],
 ['64\xa0°F',
  '64\xa0°F',
  '63\xa0°F',
  '63\xa0°F',
  '63\xa0°F',
  '63\xa0°F',
  '61\xa0°F',
  '61\xa0°F',
  '61\xa0°F',
  '61\xa0°F',
  '63\xa0°F',
  '63\xa0°F',
  '61\xa0°F',
  '61\xa0°F',
  '63\xa0°F',
  '61\xa0°F',
  '61\xa0°F',
  '61\xa0°F',
  '63\xa0°F',
  '63\xa0°F',
  '64\xa0°F',
  '64\xa0°F',
  '66\xa0°F',
  '68\xa0°F',
  '70\xa0°F',
  '70\xa0°F',

Seville save, early stop

In [341]:
dates_ =[]
temps = []
dew_points = []
humidities = []
winds = []
wind_speeds = []
wind_gusts = []
pressures = []
precips = []
conditions = []
for day_key in data_1.keys():
    for ind, t in enumerate(data_1[day_key][0]):
        dates_.append(day_key+' '+data_1[day_key][0][ind])
        temps.append(data_1[day_key][1][ind])
        dew_points.append(data_1[day_key][2][ind])
        humidities.append(data_1[day_key][3][ind])
        winds.append(data_1[day_key][4][ind])
        wind_speeds.append(data_1[day_key][5][ind])
        wind_gusts.append(data_1[day_key][6][ind])
        pressures.append(data_1[day_key][7][ind])
        precips.append(data_1[day_key][8][ind])
        conditions.append(data_1[day_key][9][ind])
bilbao_2018_05_18 = pd.DataFrame({'date':dates_,
              'temp':temps,
              'dew_point':dew_points,
              'humidities': humidities,
              'wind':winds,
              'wind_speeds':wind_speeds,
              'pressures':pressures,
              'precips':precips,
              'condition':conditions})
bilbao_2018_05_18.to_csv('../data/bilbao_2018_05_18.csv')

In [298]:
dates_ =[]
temps = []
dew_points = []
humidities = []
winds = []
wind_speeds = []
wind_gusts = []
pressures = []
precips = []
conditions = []
for city_key in city_data.keys():
    for day_key in city_data[city_key].keys():
        for ind, t in enumerate(city_data[city_key][day_key][0]):
            dates_.append(day_key+' '+city_data[city_key][day_key][0][ind])
            temps.append(city_data[city_key][day_key][1][ind])
            dew_points.append(city_data[city_key][day_key][2][ind])
            humidities.append(city_data[city_key][day_key][3][ind])
            winds.append(city_data[city_key][day_key][4][ind])
            wind_speeds.append(city_data[city_key][day_key][5][ind])
            wind_gusts.append(city_data[city_key][day_key][6][ind])
            pressures.append(city_data[city_key][day_key][7][ind])
            precips.append(city_data[city_key][day_key][8][ind])
            conditions.append(city_data[city_key][day_key][9][ind])
madrid_next = pd.DataFrame({'date':dates_,
              'temp':temps,
              'dew_point':dew_points,
              'humidities': humidities,
              'wind':winds,
              'wind_speeds':wind_speeds,
              'pressures':pressures,
              'precips':precips,
              'condition':conditions})
madrid_next

,date,temp,dew_point,humidities,wind,wind_speeds,pressures,precips,condition
0,2021-09-16 11:00 PM,64 °F,57 °F,77 °%,WSW,3 °mph,27.95 °in,0.0 °in,Fair
1,2021-09-16 11:30 PM,64 °F,57 °F,77 °%,WSW,5 °mph,27.95 °in,0.0 °in,Fair
2,2021-09-16 12:00 AM,63 °F,57 °F,82 °%,WSW,2 °mph,27.95 °in,0.0 °in,Fair
3,2021-09-16 12:30 AM,63 °F,57 °F,82 °%,WSW,2 °mph,27.95 °in,0.0 °in,Fair
4,2021-09-16 1:00 AM,63 °F,57 °F,82 °%,WSW,2 °mph,27.95 °in,0.0 °in,Fair
...,...,...,...,...,...,...,...,...,...
5205,2021-12-31 9:30 PM,45 °F,41 °F,87 °%,NNW,5 °mph,28.28 °in,0.0 °in,Fair
5206,2021-12-31 10:00 PM,43 °F,39 °F,87 °%,NNW,3 °mph,28.28 °in,0.0 °in,Fair
5207,2021-12-31 10:30 PM,41 °F,41 °F,100 °%,N,3 °mph,28.28 °in,0.0 °in,Fair
5208,2021-12-31 11:00 PM,41 °F,39 °F,93 °%,N,3 °mph,28.28 °in,0.0 °in,Fair


In [293]:
len(winds)

1070

In [112]:
dates_stamp = pd.date_range(start=dt.datetime(2015,1,1), end=dt.datetime(2021,12,31),freq='D')
dates = [str(x)[:10] for x in dates_stamp]

data = {}

for date in dates:
    # Request page
    url = str(str(page) + str(date))
    r = render_page(url)
    soup = BS(r, "html.parser")
    container = soup.find('lib-city-history-observation')
    table = container.find('tbody')

    # Extract table from page and add it to data
    data[date] = extract_table(table)


AttributeError: 'NoneType' object has no attribute 'find_all'

In [223]:
data.keys()

dict_keys(['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-

In [219]:
df = pd.DataFrame()
dates_ =[]
temps = []
dew_points = []
humidities = []
winds = []
wind_speeds = []
wind_gusts = []
pressures = []
precips = []
conditions = []
for key in data.keys():
    for ind, t in enumerate(data[key][0]):
        dates_.append(key+' '+data[key][0][ind])
        temps.append(data[key][1][ind])
        dew_points.append(data[key][2][ind])
        humidities.append(data[key][3][ind])
        winds.append(data[key][4][ind])
        wind_speeds.append(data[key][5][ind])
        wind_gusts.append(data[key][6][ind])
        pressures.append(data[key][7][ind])
        precips.append(data[key][8][ind])
        conditions.append(data[key][9][ind])
madrid_next = pd.DataFrame({'date':dates_,
              'temp':temps,
              'dew_point':dew_points,
              'humidities': humidities,
              'wind':winds,
              'wind_speeds':wind_speeds,
              'pressures':pressures,
              'precips':precips,
              'condition':conditions})
    

In [266]:
city_data['LEMD']['2021-12-31']

[['12:00 AM',
  '12:30 AM',
  '1:00 AM',
  '1:30 AM',
  '2:00 AM',
  '2:30 AM',
  '3:00 AM',
  '3:30 AM',
  '4:00 AM',
  '4:30 AM',
  '5:00 AM',
  '5:30 AM',
  '6:00 AM',
  '6:30 AM',
  '7:00 AM',
  '7:30 AM',
  '8:00 AM',
  '8:30 AM',
  '9:00 AM',
  '9:30 AM',
  '10:00 AM',
  '10:30 AM',
  '11:00 AM',
  '11:30 AM',
  '12:00 PM',
  '12:30 PM',
  '1:00 PM',
  '1:30 PM',
  '2:00 PM',
  '2:30 PM',
  '3:00 PM',
  '3:30 PM',
  '4:00 PM',
  '4:30 PM',
  '5:00 PM',
  '5:30 PM',
  '6:00 PM',
  '6:30 PM',
  '7:00 PM',
  '7:30 PM',
  '8:00 PM',
  '8:30 PM',
  '9:00 PM',
  '9:30 PM',
  '10:00 PM',
  '10:30 PM',
  '11:00 PM',
  '11:30 PM'],
 ['43\xa0°F',
  '41\xa0°F',
  '41\xa0°F',
  '39\xa0°F',
  '39\xa0°F',
  '39\xa0°F',
  '37\xa0°F',
  '37\xa0°F',
  '36\xa0°F',
  '36\xa0°F',
  '36\xa0°F',
  '36\xa0°F',
  '37\xa0°F',
  '34\xa0°F',
  '34\xa0°F',
  '34\xa0°F',
  '34\xa0°F',
  '34\xa0°F',
  '34\xa0°F',
  '36\xa0°F',
  '37\xa0°F',
  '39\xa0°F',
  '41\xa0°F',
  '45\xa0°F',
  '48\xa0°F',
  '50\xa0°F',

In [287]:
len(madrid_next)

1070

In [299]:
madrid_2 = pd.concat([madrid, madrid_next])

In [300]:
madrid_2['top_of_hour'] = madrid_2.date.apply(lambda x: x[-5]=='0')
madrid_3 = madrid_2.loc[madrid_2.top_of_hour].copy()
madrid_3

,date,temp,dew_point,humidities,wind,wind_speeds,pressures,precips,condition,top_of_hour
0,2015-01-01 12:00 AM,30 °F,25 °F,80 °%,S,2 °mph,28.40 °in,0.0 °in,Fair,True
2,2015-01-01 1:00 AM,30 °F,25 °F,80 °%,SW,3 °mph,28.43 °in,0.0 °in,Fair,True
4,2015-01-01 2:00 AM,27 °F,25 °F,93 °%,SSE,2 °mph,28.43 °in,0.0 °in,Fair,True
6,2015-01-01 3:00 AM,27 °F,27 °F,100 °%,VAR,1 °mph,28.43 °in,0.0 °in,Fair,True
8,2015-01-01 4:00 AM,25 °F,25 °F,100 °%,CALM,0 °mph,28.43 °in,0.0 °in,Fair,True
...,...,...,...,...,...,...,...,...,...,...
5200,2021-12-31 7:00 PM,48 °F,45 °F,87 °%,N,3 °mph,28.22 °in,0.0 °in,Fair,True
5202,2021-12-31 8:00 PM,46 °F,43 °F,87 °%,VAR,2 °mph,28.25 °in,0.0 °in,Fair,True
5204,2021-12-31 9:00 PM,46 °F,43 °F,87 °%,N,3 °mph,28.25 °in,0.0 °in,Fair,True
5206,2021-12-31 10:00 PM,43 °F,39 °F,87 °%,NNW,3 °mph,28.28 °in,0.0 °in,Fair,True


In [302]:
madrid_3.to_csv('../data/madrid_pulled.csv')

In [259]:
y = pd.DataFrame(index=pd.date_range(start=dt.datetime(2015,1,1), end=dt.datetime(2021,12,31), freq='H'))
y_2 = y.join(madrid_3, how='left')

TypeError: '<' not supported between instances of 'int' and 'Timestamp'

In [203]:
madrid['top_of_hour'] = madrid.date.apply(lambda x: x[-5]=='0')

In [206]:
madrid.loc[madrid.top_of_hour]

,date,temp,dew_point,humidities,wind,wind_speeds,pressures,precips,condition,top_of_hour
0,2015-01-01 12:00 AM,30 °F,25 °F,80 °%,S,2 °mph,28.40 °in,0.0 °in,Fair,True
2,2015-01-01 1:00 AM,30 °F,25 °F,80 °%,SW,3 °mph,28.43 °in,0.0 °in,Fair,True
4,2015-01-01 2:00 AM,27 °F,25 °F,93 °%,SSE,2 °mph,28.43 °in,0.0 °in,Fair,True
6,2015-01-01 3:00 AM,27 °F,27 °F,100 °%,VAR,1 °mph,28.43 °in,0.0 °in,Fair,True
8,2015-01-01 4:00 AM,25 °F,25 °F,100 °%,CALM,0 °mph,28.43 °in,0.0 °in,Fair,True
...,...,...,...,...,...,...,...,...,...,...
118541,2021-09-16 6:00 PM,75 °F,52 °F,44 °%,N,7 °mph,27.95 °in,0.0 °in,Fair,True
118543,2021-09-16 7:00 PM,73 °F,52 °F,47 °%,N,3 °mph,27.95 °in,0.0 °in,Fair,True
118545,2021-09-16 8:00 PM,72 °F,55 °F,57 °%,N,3 °mph,27.95 °in,0.0 °in,Fair,True
118547,2021-09-16 9:00 PM,70 °F,55 °F,60 °%,N,3 °mph,27.98 °in,0.0 °in,Fair,True


In [208]:
madrid.to_csv('../data/madrid_pulled.csv')

In [60]:
row = check.find_all('tr', class_='ng-star-inserted')[0]
cols = len(row.find_all('td', class_='ng-star-inserted'))
cols

10

In [4]:
dates = ['2015-1-1','2015-1-2']
page = 'https://www.wunderground.com/history/daily/LEMD/date/'
df_output = scraper(page,dates)

Data not in normal length


UnboundLocalError: local variable 'Date' referenced before assignment

In [3]:
madrid = Point(40.4168, 3.7038, 70)
madrid

In [11]:
start = dt.datetime(2015,1,1)
stop = dt.datetime(2015,1,2)

In [13]:
data = Hourly(21230, start = start, end=stop)

In [10]:
data.fetch()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,


In [ ]:
endpoint = 'https://bulk.meteostat.net/v2/hourly/obs/{station}.csv.gz'

In [ ]:
# Initialize dictionary to hold info
data = {'values':[], 'datetime':[]}

# Initialize request parameters
widget='precios-mercados-tiempo-real'
lang='en'
category='mercados'
time_trunc = 'hour'

# Request start and stop dates
start = dt.datetime(2019,1,1)
stop_pull = dt.datetime(2022,1,20)

while start < stop_pull:
    
    # calculate the stop time
    stop = start + dt.timedelta(hours=372)
    
    # if the stop time is at the end of the pull range, set it equal to last day
    if stop >= stop_pull:
        stop = stop_pull
    
    # Reformat date for api pull
    start_date = f'{start}'.replace(' ', 'T')[:-3]
    stop_date = f'{stop}'.replace(' ', 'T')[:-3]
    
    # Create get request
    url = f'https://apidatos.ree.es/{lang}/datos/{category}/{widget}?start_date={start_date}&end_date={stop_date}&time_trunc={time_trunc}'
    request = requests.get(url)
    json = request.json()
    
    # store request in data dictionary
    for val in json['included'][0]['attributes']['values']:
        data['values'].append(val['value'])
        data['datetime'].append(val['datetime'])
    
    # Go to the next hour
    start = stop + dt.timedelta(hours=1)
    
# Convert data to df
df = pd.DataFrame({'datetime':data['datetime'], 'price actual':data['values']})
df['datetime'] = pd.to_datetime(df['datetime'].apply(lambda x: x[:-16]))
df.set_index('datetime', inplace=True)
df.to_csv('../data/prices.csv')